In [ ]:
import os
import pandas as pd
import json


from collections import defaultdict


# All this file does it takes the json files and exports the csv file. Not really much to see here

In [ ]:
current = os.getcwd() #current directory
data_folder = '/CORD-19-research-challenge/'

#The data folders
folders = ['document_parses']

#Counting the files
paths = dict()
for folder in folders:
    f = current + data_folder + folder + '/pdf_json/'
    filenames = os.listdir(f)
    paths[folder] = filenames

for key in paths.keys():
    print("Number of articles retrieved from {}:".format(key), len(paths[key]))

#loading the data from the paths into a list of json text data as dict
def load(folder:str) -> list:
    all_files = []
    for filename in paths[folder]:
        file = os.path.join(current + data_folder + folder + '/pdf_json/' + filename)
        f = json.load(open(file,'rb'))
        all_files.append(f)
    return all_files

#loaded = ['biorxiv_medrxiv','comm_use_subset','custom_license','noncomm_use_subset','arxiv']

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_authors(authors):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        
        name_ls.append(name)
    
    return ", ".join(name_ls)

#Take the json, format all the data we need and place into dataframe
def finish(data):
    clean = []
    for f in data:
        features = [
            f['paper_id'],
            f['metadata']['title'],
            format_authors(f['metadata']['authors']),
            format_body(f['abstract']),
            format_body(f['body_text']),
        ]
        clean.append(features)

    column_names = [
        'paper_id',
        'title',
        'authors',
        'abstract',
        'body_text'
    ]

    df = pd.DataFrame(clean,columns=column_names)
    return df

# bio = load('biorxiv_medrxiv')
# print('loaded bio')
# comm_use_subset = load('comm_use_subset')
# print('loaded comm_use_subset')

# custom_license = load('custom_license')
# print('loaded custom_license')

# noncomm_use_subset = load('noncomm_use_subset')
# print('loaded noncomm_use_subset')

# arxiv = load('arxiv')
# print('loaded arxiv')

art = load('document_parses')
print('loaded documents')

# b = finish(bio)
# print('finshed bio')

# comm = finish(comm_use_subset)
# print('finshed comm_use_subset')

# custom = finish(custom_license)
# print('finshed custom_license')

# n = finish(noncomm_use_subset)
# print('finshed noncomm_use_subset')

# a = finish(arxiv)
# print('finished arxiv')

#Create combined dataset
big = finish(art)
print("finshed")

big.to_csv('big.csv',index=False)